# Clash of Clans Automation

This is a work in progress to automate the clash of clans game.
You'll need:
* Windows
* Bluestacks app: https://www.bluestacks.com/download.html

## Grab screen

Got the script from pygta5 sentdex: https://github.com/Sentdex/pygta5/blob/master/grabscreen.py

In [1]:
# Done by Frannecklp
import cv2
import numpy as np
import win32gui
import win32ui
import win32con
import win32api

In [2]:
def grab_screen(region=None):
    hwin = win32gui.GetDesktopWindow()

    if region:
        left, top, x2, y2 = region
        width = x2 - left + 1
        height = y2 - top + 1
    else:
        width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
        height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
        left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
        top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc)
    memdc = srcdc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)

    signedIntsArray = bmp.GetBitmapBits(True)
    img = np.fromstring(signedIntsArray, dtype='uint8')
    img.shape = (height, width, 4)

    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())

    return cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

In [3]:
def show_image(image):
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
def grab_coc():
    image = grab_screen()
    # Use MPos to get the coordinates: https://sourceforge.net/projects/mpos/
    # Top-left
    # 265, 106
    # Bottom-right
    # 1718, 938
    # Add 1920 to width slices because of dual monitors
    # Crop width and height
    width = slice(2185, 3638)
    height = slice(106, 938)

    image = image[height, width]
    return image

In [24]:
show_image(image)

C:\Users\patta\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [9]:
# while True:
#     image = grab_screen()

#     cv2.imshow("Image", image)
#     cv2.waitKey(15)

C:\Users\patta\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


KeyboardInterrupt: 

## Move Mouse

In [5]:
import pyautogui
import time

In [7]:
pyautogui.moveTo(100, 150)

In [18]:
clash_of_clans_loc = (1100, 280)
okay_loc = (1050, 650)

In [56]:
def open_coc():
    """
    Open clash of clans on bluestacks.
    """
    pyautogui.moveTo(clash_of_clans_loc)
    pyautogui.click(clash_of_clans_loc)

In [57]:
def exit_game():
    """
    Exit clash of clans on bluestacks.
    """
    # Click somewhere on the app
    pyautogui.click(clash_of_clans_loc)
    # Use the bluestacks hotkey to quit
    pyautogui.hotkey('ctrl','shift','2')
    # Click on Okay
    time.sleep(0.5)
    pyautogui.click(okay_loc)

## Template matching

https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html

In [8]:
import cv2

In [9]:
import numpy as np

In [29]:
img_rgb = grab_coc()
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

C:\Users\patta\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [38]:
show_image(img_rgb)

In [31]:
template = cv2.imread('images/elixir_collector_real.png', 0)
w, h = template.shape[::-1]

In [32]:
show_image(template)

In [33]:
result = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)

In [37]:
print(loc)

(array([151, 151, 151, 152, 152, 152, 152, 152, 153, 153, 153, 153, 153,
       154, 154, 154, 154, 154, 155, 155, 155, 211, 211, 212, 212, 269,
       269, 270, 270, 540, 541], dtype=int64), array([ 813,  814,  815,  812,  813,  814,  815,  816,  812,  813,  814,
        815,  816,  812,  813,  814,  815,  816,  813,  814,  815,  400,
        401,  400,  401, 1072, 1073, 1072, 1073,  530,  530], dtype=int64))


In [39]:
# print(result)
demo_img = img_rgb.copy()
threshold = 0.5
loc = np.where(result >= threshold)
# print(loc)
for pt in zip(*loc[::-1]):
    print(pt)
    cv2.rectangle(demo_img, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
show_image(demo_img)
# cv2.imwrite('images/res.png', img_rgb)

(813, 151)
(814, 151)
(815, 151)
(812, 152)
(813, 152)
(814, 152)
(815, 152)
(816, 152)
(812, 153)
(813, 153)
(814, 153)
(815, 153)
(816, 153)
(812, 154)
(813, 154)
(814, 154)
(815, 154)
(816, 154)
(813, 155)
(814, 155)
(815, 155)
(400, 211)
(401, 211)
(400, 212)
(401, 212)
(1072, 269)
(1073, 269)
(1072, 270)
(1073, 270)
(530, 540)
(530, 541)


In [40]:
blue_stack_win_top_left = (267, 107)

In [44]:
pyautogui.moveTo(267+813+w/2, 107+151+h/2)

In [42]:
813+200

1013

In [55]:
img_rgb = grab_coc()
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

In [58]:
elixir_template = cv2.imread('images/elixir_collector_real.png', 0)
gold_template = cv2.imread('images/gold_collector_real.png', 0)
dark_elixir_template = cv2.imread('images/dark_elixir_collector_real.png', 0)

In [60]:
blue_stack_win_top_left = (267, 107)

In [59]:
def collect_resource(template):
    """
    Match the resource template and click on resource to collect it.
    """
    w, h = template.shape[::-1]
    result = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.5
    loc = np.where(result >= threshold)

    loc_pos = [pt for pt in zip(*loc[::-1])]

    resource_loc = (loc_pos[0][0] + blue_stack_win_top_left[0] + w/2, loc_pos[0][1] + blue_stack_win_top_left[1] + h/2)

    pyautogui.moveTo(resource_loc)

    pyautogui.click(resource_loc)
    time.sleep(1)

In [61]:
for template in [elixir_template, gold_template, dark_elixir_template]:
    collect_resource(template)